# 1. Data Extraction

In this notebook we will extract our data and analyse it. For that purpose, we are importing from our library where we define the
```bcrp_dataframe``` dataframe. This function will allows us to use the API interface of the Central Bank of Reserve of Peru (BCRP) to automatically create a pandas dataframe with the necessary codes.

## 1.1 Libraries

We import the necessary libraries, including our own library in the modules file

In [3]:
# Warnings
import warnings
warnings.filterwarnings("ignore")

# Basic Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import seaborn as sns
from scipy import stats
from functools import reduce

# Statsmodels
import statsmodels.api as sm
import pmdarima as pmd
from pmdarima.arima import auto_arima
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.var_model import VARResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import STL

# Machine Learning models
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.linear_model import Ridge, Lasso, ElasticNet, ElasticNetCV, LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
    median_absolute_error,
    r2_score,
    precision_score

)

from xgboost import XGBRegressor



In [4]:
# We import our own functions
import sys
sys.path.append('../../..')  # Move two levels up to the project root
from modules.functions import *

## 1.2 Extraction
We define our inputs and apply them the ```bcrp_dataframe``` function in order to obtain the pandas dataframe with the corresponding series

We define the following inputs:

    series     = the code of the series we are going to extract
    start_date = the starting date, when the BCRP starts using the interest rate as a policy measure
    end_date   = December 2019
    freq       = Monthly frequency

### df_1
We can now create the first dataframe with the ```bcrp_dataframe``` function. This dataframe contains out target variable Headline Inflation 

In [5]:
series     = ['PN01271PM']
start_date = '2003-09'
end_date   = '2019-12'
freq       = 'Mensual'

In [6]:
df_1 = bcrp_dataframe( series , start_date , end_date , freq )
df_1.head()

,Índice de precios Lima Metropolitana (var% mensual) - IPC
Fecha,
2003-09-01,0.558598
2003-10-01,0.049032
2003-11-01,0.167685
2003-12-01,0.563951
2004-01-01,0.537447


In [7]:
df_1 = get_trend(df_1)
df_1.head()

,Índice de precios Lima Metropolitana (var% mensual) - IPC
Fecha,
2003-09-01,0.518834
2003-10-01,0.492247
2003-11-01,0.465462
2003-12-01,0.438413
2004-01-01,0.411138


### df_2
We create the second dataframe with the ```bcrp_dataframe``` function. This dataframe contains rate variables. We use this variables in levels. It is not necessary to differentiate them.

In [8]:
series     = ['PD04722MM']
start_date = '2003-09'
end_date   = '2019-12'
freq       = 'Mensual'

In [9]:
df_2 = bcrp_dataframe( series , start_date , end_date , freq )
df_2.head()

,Tasas de interés del Banco Central de Reserva - Tasa de Referencia de la Política Monetaria
Fecha,
2003-09-01,2.75
2003-10-01,2.75
2003-11-01,2.50
2003-12-01,2.50
2004-01-01,2.50


### df_3
We create the third dataframe with the ```bcrp_dataframe``` function. This dataframe contains monetary variables as well as commodities. We differentiate those variables in order to get their monthly % change

In [10]:
series     = ['PN00495MM', 'PN06481IM', 'PN02125PM', 'PN01661XM','PN01662XM','PN01664XM','PN01660XM']
start_date = '2003-09'
end_date   = '2019-12'
freq       = 'Mensual'

In [13]:
df_3 = bcrp_dataframe( series , start_date , end_date , freq )
df_3 = np.log(df_3)
df_3 = df_3.dropna()
df_3.head()

,Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado,Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$),Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100),Cotizaciones de productos (promedio del periodo) - Trigo - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Maíz - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Aceite Soya - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Petróleo - WTI (US$ por barriles)
Fecha,,,,,,,
2003-09-01,8.591877,9.185581,5.440117,4.884206,4.419001,6.238332,3.348787
2003-10-01,8.602051,9.191108,5.495508,4.900658,4.390800,6.404890,3.412510
2003-11-01,8.613430,9.240173,5.493832,5.009696,4.463461,6.421768,3.434971
2003-12-01,8.629901,9.229584,5.488208,5.046169,4.511148,6.471752,3.472088
2004-01-01,8.649204,9.265177,5.482848,5.049709,4.571827,6.498180,3.531787


In [14]:
df = df_1.join(df_2).join(df_3)
df.dropna(inplace=True)
df.head()

,Índice de precios Lima Metropolitana (var% mensual) - IPC,Tasas de interés del Banco Central de Reserva - Tasa de Referencia de la Política Monetaria,Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado,Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$),Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100),Cotizaciones de productos (promedio del periodo) - Trigo - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Maíz - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Aceite Soya - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Petróleo - WTI (US$ por barriles)
Fecha,,,,,,,,,
2003-09-01,0.518834,2.75,8.591877,9.185581,5.440117,4.884206,4.419001,6.238332,3.348787
2003-10-01,0.492247,2.75,8.602051,9.191108,5.495508,4.900658,4.390800,6.404890,3.412510
2003-11-01,0.465462,2.50,8.613430,9.240173,5.493832,5.009696,4.463461,6.421768,3.434971
2003-12-01,0.438413,2.50,8.629901,9.229584,5.488208,5.046169,4.511148,6.471752,3.472088
2004-01-01,0.411138,2.50,8.649204,9.265177,5.482848,5.049709,4.571827,6.498180,3.531787


In [15]:
df.tail()

,Índice de precios Lima Metropolitana (var% mensual) - IPC,Tasas de interés del Banco Central de Reserva - Tasa de Referencia de la Política Monetaria,Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado,Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$),Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100),Cotizaciones de productos (promedio del periodo) - Trigo - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Maíz - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Aceite Soya - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Petróleo - WTI (US$ por barriles)
Fecha,,,,,,,,,
2019-08-01,0.155238,2.50,10.783622,11.131211,5.748980,5.194315,4.959057,6.443189,4.003798
2019-09-01,0.151966,2.50,10.792065,11.125202,5.748917,5.165015,4.913953,6.462062,4.040550
2019-10-01,0.148757,2.50,10.795930,11.127057,5.747810,5.204847,4.978704,6.503629,3.988299
2019-11-01,0.145714,2.25,10.802030,11.109024,5.746721,5.274724,4.942720,6.529491,4.044437
2019-12-01,0.142886,2.25,10.813291,11.131899,5.744579,5.306730,4.950479,6.585121,4.092312


## 1.3 Data Inspection
We inspect the df. We first verify that all values are non-null. The, we apply the ```describe``` function to see the main variables.

In [16]:
df.isna().sum()

Índice de precios Lima Metropolitana (var% mensual) - IPC                                       0
Tasas de interés del Banco Central de Reserva  - Tasa de Referencia de la Política Monetaria    0
Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado                  0
Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$)           0
Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100)                      0
Cotizaciones de productos (promedio del periodo) - Trigo - EEUU (US$ por toneladas)             0
Cotizaciones de productos (promedio del periodo) - Maíz - EEUU (US$ por toneladas)              0
Cotizaciones de productos (promedio del periodo) - Aceite Soya - EEUU (US$ por toneladas)       0
Cotizaciones de productos (promedio del periodo) - Petróleo - WTI (US$ por barriles)            0
dtype: int64

In [17]:
df.describe()

,Índice de precios Lima Metropolitana (var% mensual) - IPC,Tasas de interés del Banco Central de Reserva - Tasa de Referencia de la Política Monetaria,Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado,Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$),Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100),Cotizaciones de productos (promedio del periodo) - Trigo - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Maíz - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Aceite Soya - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Petróleo - WTI (US$ por barriles)
count,196.000000,196.000000,196.000000,196.000000,196.000000,196.000000,196.000000,196.000000,196.000000
mean,0.237890,3.665816,9.967408,10.529348,5.597523,5.367305,4.954970,6.625654,4.182633
std,0.106886,1.106475,0.659690,0.629997,0.106667,0.284016,0.380062,0.272111,0.339344
min,0.056201,1.250000,8.591877,9.185581,5.439158,4.792837,4.164457,6.106831,3.348787
25%,0.156165,2.937500,9.449695,10.113836,5.502205,5.129676,4.799998,6.442861,3.932685
50%,0.241131,3.750000,10.131215,10.794583,5.606932,5.350658,4.913894,6.584073,4.161373
75%,0.293527,4.250000,10.547071,11.040132,5.695481,5.601870,5.132748,6.811396,4.484188
max,0.518834,6.500000,10.813291,11.131899,5.777829,6.077078,5.757876,7.232960,4.897019


We have 195 observation ranging from ```2003-10-01``` to ```2019-12-01```. The mean of monthly % change of all CPI variables is around 0.2. The mean of the lacing rate and the interest rate is 10.7% and 3.67%, respectively. The three monetary variables have a small monthly % change, around 0.01 and 0.001 for Minimum Wage index. 

## 1.4 Data adjustment
We will rename the columns for easier identification of the variables. We will also create a new dataframe with the lags of the variables. 

In [18]:
# New column names
columns = {
    'Índice de precios Lima Metropolitana (var% mensual) - IPC': 'CPI',
    'Tasas de interés del Banco Central de Reserva  - Tasa de Referencia de la Política Monetaria': 'Monetary Policy Rate',
    'Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado': 'Circulating Currency Seasonally Adjusted (mill S/)',
    'Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$)': 'Net International Reserves (mill $)',
    'Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100)': 'Real Minimum Wage (Index)',
    'Cotizaciones de productos (promedio del periodo) - Trigo - EEUU (US$ por toneladas)': 'Wheat (US$ per ton)',
    'Cotizaciones de productos (promedio del periodo) - Maíz - EEUU (US$ por toneladas)': 'Corn  (US$ per ton)',
    'Cotizaciones de productos (promedio del periodo) - Aceite Soya - EEUU (US$ por toneladas)': 'Soybean oil (US$ per ton)',
    'Cotizaciones de productos (promedio del periodo) - Petróleo - WTI (US$ por barriles)': 'Crude oil (US$ per barrel)'  
}

# We rename the columns so they are easier to analyse
df.rename(columns=columns, inplace=True)

In [19]:
df_lags = df.copy()

for variable in df_lags.columns[1:]:
    df_lags[f'{variable}_lag_1'] = df_lags[variable].shift()
    df_lags[f'{variable}_lag_2'] = df_lags[variable].shift(2)
    df_lags[f'{variable}_lag_3'] = df_lags[variable].shift(3)
    df_lags[f'{variable}_lag_4'] = df_lags[variable].shift(4)

In [20]:
# We delete contemporary variables
df_lags.drop(columns = ['Monetary Policy Rate','Circulating Currency Seasonally Adjusted (mill S/)',
       'Net International Reserves (mill $)', 'Real Minimum Wage (Index)', 'Wheat (US$ per ton)', 'Corn  (US$ per ton)', 
       'Soybean oil (US$ per ton)', 'Crude oil (US$ per barrel)'], inplace = True)

df_lags = df_lags.dropna()

## 1.5 Save Results
We save it to the ```input``` folder, where we can use it to do the forecasting in the next notebook.

In [21]:
df.to_csv('../../../input/df_raw_test.csv')

In [22]:
df_lags.to_csv('../../../input/df_lags_test.csv')

In [23]:
df_lags.tail()

,CPI,Monetary Policy Rate_lag_1,Monetary Policy Rate_lag_2,Monetary Policy Rate_lag_3,Monetary Policy Rate_lag_4,Circulating Currency Seasonally Adjusted (mill S/)_lag_1,Circulating Currency Seasonally Adjusted (mill S/)_lag_2,Circulating Currency Seasonally Adjusted (mill S/)_lag_3,Circulating Currency Seasonally Adjusted (mill S/)_lag_4,Net International Reserves (mill $)_lag_1,...,Corn (US$ per ton)_lag_3,Corn (US$ per ton)_lag_4,Soybean oil (US$ per ton)_lag_1,Soybean oil (US$ per ton)_lag_2,Soybean oil (US$ per ton)_lag_3,Soybean oil (US$ per ton)_lag_4,Crude oil (US$ per barrel)_lag_1,Crude oil (US$ per barrel)_lag_2,Crude oil (US$ per barrel)_lag_3,Crude oil (US$ per barrel)_lag_4
Fecha,,,,,,,,,,,,,,,,,,,,,
2019-08-01,0.155238,2.75,2.75,2.75,2.75,10.775826,10.772159,10.761742,10.762972,11.108755,...,4.928328,4.861634,6.426634,6.420082,6.378261,6.419762,4.048862,4.001086,4.106295,4.156843
2019-09-01,0.151966,2.50,2.75,2.75,2.75,10.783622,10.775826,10.772159,10.761742,11.131211,...,5.083703,4.928328,6.443189,6.426634,6.420082,6.378261,4.003798,4.048862,4.001086,4.106295
2019-10-01,0.148757,2.50,2.50,2.75,2.75,10.792065,10.783622,10.775826,10.772159,11.125202,...,5.095125,5.083703,6.462062,6.443189,6.426634,6.420082,4.040550,4.003798,4.048862,4.001086
2019-11-01,0.145714,2.50,2.50,2.50,2.75,10.795930,10.792065,10.783622,10.775826,11.127057,...,4.959057,5.095125,6.503629,6.462062,6.443189,6.426634,3.988299,4.040550,4.003798,4.048862
2019-12-01,0.142886,2.25,2.50,2.50,2.50,10.802030,10.795930,10.792065,10.783622,11.109024,...,4.913953,4.959057,6.529491,6.503629,6.462062,6.443189,4.044437,3.988299,4.040550,4.003798


In [25]:
df.tail()

,CPI,Monetary Policy Rate,Circulating Currency Seasonally Adjusted (mill S/),Net International Reserves (mill $),Real Minimum Wage (Index),Wheat (US$ per ton),Corn (US$ per ton),Soybean oil (US$ per ton),Crude oil (US$ per barrel)
Fecha,,,,,,,,,
2019-08-01,0.155238,2.50,10.783622,11.131211,5.748980,5.194315,4.959057,6.443189,4.003798
2019-09-01,0.151966,2.50,10.792065,11.125202,5.748917,5.165015,4.913953,6.462062,4.040550
2019-10-01,0.148757,2.50,10.795930,11.127057,5.747810,5.204847,4.978704,6.503629,3.988299
2019-11-01,0.145714,2.25,10.802030,11.109024,5.746721,5.274724,4.942720,6.529491,4.044437
2019-12-01,0.142886,2.25,10.813291,11.131899,5.744579,5.306730,4.950479,6.585121,4.092312
